In [ ]:
import os, sys
dir1 = os.path.abspath('..\..')
sys.path.append(dir1+ "\\")

import MetaTrader5 as mt
from datetime import datetime as dt

# custom
from my_jupyter.user import User
from my_jupyter.strategy_manager import StrategyManager
from my_jupyter.operation import Operation
from my_jupyter.strategies.min_max_strategy import MinMaxStrategy
from my_jupyter.market_data_repository import MarketDataRepository
from my_jupyter.filters.moving_average import MovingAverageFilter
from my_jupyter.filters.consequent_limits_counter import ConsequentLimitsCounterFilter
from my_jupyter.filters.directioned_bars_filter import DirectionedBarsFilter

# from Mbox import Mbox
# from boleta_1 import gerar_boleta
# from gerenciamento import Gerenciamento

## GUI's
from daemon_tools import DaemonThreadMini


In [ ]:


filters = [DirectionedBarsFilter(candles=5), MovingAverageFilter(), MovingAverageFilter(period=200)]
strategies = [MinMaxStrategy(filters=filters)]
operations = [Operation("WINV23", 1, mt.TIMEFRAME_M1, can_buy=True, can_sell=True)]
user = User(strategies=strategies, operations=operations, trading_time_start=dt(2023,10,5,9,5,0))
market_data_repository = MarketDataRepository()
strategy = StrategyManager(market_data_repository, user)

# PARAMETROS MAIN
kwargs = {
    "strategy": strategy,
}


def main(**kwargs):
    _shift = 0
    strategy: StrategyManager = kwargs["strategy"]
    strategy.run_strategies()
    # print(f"{dt.now().strftime('%d/%m/%Y, %H:%M:%S')} >>> D-{_shift}")
def exec_daemon(**kwargs):
    main(**kwargs)
    # alarme.ring_alarme()

In [ ]:
# gerenciamento = Gerenciamento(mt, timeframe, timeframes_dict)

try:
    main(**kwargs)
except Exception as e:
    print(e.with_traceback(__tb=None))
    # Mbox.Alerta("ERRO", "busca de sinais com erro")


In [ ]:
# from my_jupyter.daemons.daemon_tools import DaemonThreadMini


# thread_exec = DaemonThreadMini(exec_daemon, seconds=0, mseconds=800, **kwargs)
# thread_exec.start()

In [1]:
import os, sys

dir1 = os.path.abspath("..\..")

if dir1 not in sys.path:
    sys.path.append(dir1 + "\\")
from my_jupyter.indicator.bars_trend_quality_ml import BarsTrendQualityML
from my_jupyter.market_data_repository import MarketDataRepository
from datetime import datetime as dt 
import pandas as pd 
# mt_rep = MarketDataRepository()
# ohlc = mt_rep.read_data("WINV23",mt_rep.mt.TIMEFRAME_M1,6)
# ohlc = mt_rep.mt.copy_rates_from_pos("WINV23", mt_rep.mt.TIMEFRAME_M1, 0, 99999)
# ohlc_0_based = ohlc[::-1]

# from_file  = "dataset\WINV23M1.csv"
# ohlc_df = pd.DataFrame(ohlc)
bars_trend_ml = BarsTrendQualityML()
bars_trend_ml.set_machine_learning_model(from_stock_market_data="WIN$")


c:\repo\python\financial-api\my_jupyter\modules\backtest_module.py:29: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(
c:\repo\python\financial-api\my_jupyter\modules\backtest_module.py:144: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  curr_min = curr[-1]


1167 1075
Epoch 1/10
84/84 [==============================] - 2s 8ms/step - loss: 1.0587 - accuracy: 0.4132 - val_loss: 0.9904 - val_accuracy: 0.7023
Epoch 2/10
84/84 [==============================] - 0s 3ms/step - loss: 0.8463 - accuracy: 0.7023 - val_loss: 0.6619 - val_accuracy: 0.7458
Epoch 3/10
84/84 [==============================] - 0s 3ms/step - loss: 0.5960 - accuracy: 0.8126 - val_loss: 0.5386 - val_accuracy: 0.8227
Epoch 4/10
84/84 [==============================] - 0s 4ms/step - loss: 0.5062 - accuracy: 0.8096 - val_loss: 0.4741 - val_accuracy: 0.8261
Epoch 5/10
84/84 [==============================] - 0s 4ms/step - loss: 0.4698 - accuracy: 0.8148 - val_loss: 0.4598 - val_accuracy: 0.8194
Epoch 6/10
84/84 [==============================] - 0s 5ms/step - loss: 0.4530 - accuracy: 0.8115 - val_loss: 0.4468 - val_accuracy: 0.8294
Epoch 7/10
84/84 [==============================] - 0s 4ms/step - loss: 0.4423 - accuracy: 0.8182 - val_loss: 0.4410 - val_accuracy: 0.8328
Epoch 8/10

<Axes: >

In [2]:
from my_jupyter.market_data_repository import MarketDataRepository
from datetime import datetime as dt 
mt_rep = MarketDataRepository()
ohlc = mt_rep.read_data("WINV23",mt_rep.mt.TIMEFRAME_M1,6)
ohlc = mt_rep.mt.copy_rates_from_pos("WINV23", mt_rep.mt.TIMEFRAME_M1, 0, 60)
ohlc_0_based = ohlc[::-1]
# print([ dt.fromtimestamp(i[0]) for i in ohlc_0_based])
# bars_trend_ml.output(ohlc_0_based[1:])
import numpy as np
# import logging
# logging.Logger.setLevel(logging.INFO)
ohlc_transform = [ohlc_0_based[i:i+5]["close"] for i in range(1, len(ohlc_0_based)-5)]
# ohlc_transform = [ i - min(i) for i in ohlc_transform ]
ohlc_transform = [ i - i[-1]  for i in ohlc_transform ]
print(ohlc_0_based[1], ohlc_transform[1])
for i in range(0,len(ohlc_transform)):
    print(">>>>>>>", np.round(bars_trend_ml.output(ohlc_transform[i])*100), dt.fromtimestamp(ohlc_0_based[i]["time"]))


(1697046840, 117385., 117415., 117340., 117375., 386, 5, 5495) [65. 50. 55. 35.  0.]


c:\repo\python\financial-api\venv\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 155ms/step
>>>>>>> [[12. 82. 58.]] 2023-10-11 15:01:00
1/1 [==============================] - 0s 43ms/step
>>>>>>> [[ 4. 83. 83.]] 2023-10-11 14:54:00


c:\repo\python\financial-api\venv\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\repo\python\financial-api\venv\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 34ms/step
>>>>>>> [[ 4. 82. 83.]] 2023-10-11 14:53:00
1/1 [==============================] - 0s 31ms/step
>>>>>>> [[ 3. 82. 87.]] 2023-10-11 14:52:00


...

1/1 [==============================] - 0s 81ms/step
>>>>>>> [[ 2. 83. 92.]] 2023-10-11 14:04:00
1/1 [==============================] - ETA: 0s

c:\repo\python\financial-api\venv\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 71ms/step
>>>>>>> [[ 2. 82. 90.]] 2023-10-11 14:03:00
1/1 [==============================] - ETA: 0s

c:\repo\python\financial-api\venv\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 68ms/step
>>>>>>> [[10. 81. 65.]] 2023-10-11 14:02:00
